[homework question link](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/homework.md)

# Q1. Getting the embeddings model

```bash
pip install -U sentence-transformers
pip install tqdm
pip install pandas
```

In [2]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

0.078222655

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_docs.append(doc)

In [6]:
ml_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [7]:
len(ml_docs)

375

# Q2. Creating the embeddings

In [8]:
import numpy as np

In [9]:
embeddings = []
for doc in ml_docs:
    qa_text = doc['question'] + " " + doc['text']
    embeddings.append(embedding_model.encode(qa_text))

In [10]:
X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search

In [11]:
v = embedding_model.encode(user_question)
v.shape

(768,)

In [11]:
scores = X.dot(v)

In [12]:
scores.shape

(375,)

- `v` is a vector with 768 tokens.
- `X` is a matrix which shows all the 768 tockes inside 375 record.
- `scores` shows the similarity of v with each of the records.

it is called the similarity between a query vector and all the embeddings.

In [13]:
max(scores)

0.6506573

In [22]:
class BuiltInVectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = BuiltInVectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

# Q4. Hit-rate for our search engine

In [23]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [24]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [29]:
from tqdm.auto import tqdm

def metric_evaluator(metric):
    def evaluate(ground_truth, search_function):
        relevance_total = []

        for q in tqdm(ground_truth):
            results = search_function(q)
            relevance = [d['id'] == q['document'] for d in results]
            relevance_total.append(relevance)
        
        return metric(relevance_total)
    
    return evaluate

In [30]:
def searcher(embeding_model, vector_search_engine):
    def search(query, num_results=10):
        vector_query = embeding_model.encode(query)
        return vector_search_engine.search(vector_query, num_results)
    return search

In [31]:
search_engine = BuiltInVectorSearchEngine(documents=documents, embeddings=X)
search_function = searcher(embedding_model, search_engine)

hit_rate_evaluator = metric_evaluator(hit_rate)
hit_rate_evaluator(ground_truth[:10], lambda v: search_function(v['question'], num_results=5))

100%|██████████| 10/10 [00:00<00:00, 16.16it/s]


0.0

# Q5. Indexing with Elasticsearch

```bash
docker run --rm \
    --name elastic \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [20]:
v.shape

(768,)

## 5.1. create the index

In [15]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
index_name = "course-questions"

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## 5.2. index the documents

In [23]:
for i, doc in tqdm(enumerate(ml_docs)):
    doc['question_text_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)

375it [00:10, 34.85it/s]


In [24]:
es_client.count(index=index_name)

ObjectApiResponse({'count': 375, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [26]:
es_client.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        },
        "size": 5
    }
)

ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 375, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'course-questions', '_id': 'CopTxpAB65ah-_iDM-Rk', '_score': 1.0, '_source': {'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I sign up?', 'course': 'machine-learning-zoomcamp', 'id': '0227b872', 'question_text_vector': [0.08805905282497406, 0.01559354830533266, 0.07925581932067871, 0.025275787338614464, 0.07557650655508041, -0.03905969485640526, -0.04138

In [17]:
query = {
    "field": "question_text_vector",
    "query_vector": v,
    "k": 1,
    "num_candidates": 10000, 
}

res = es_client.search(
    index=index_name,
    knn=query,
    source=["id"]
)

for doc in res['hits']['hits']:
    print(doc['_score'], doc['_source'])

0.82532895 {'id': 'ee58a693'}


# Q6. Hit-rate for Elasticsearch

In [21]:
class ElasticSearchVectorSearchEngine():
    def __init__(self, es_client):
        self.es_client = es_client

    def search(self, v_query, num_results=10):
        query = {
            "field": "question_text_vector",
            "query_vector": v_query,
            "k": num_results,
            "num_candidates": 10000, 
        }

        res = self.es_client.search(
            index=index_name,
            knn=query,
            source=['text', 'section', 'question', 'id']
        )
        return list(map(lambda x: x['_source'], res['hits']['hits']))

search_engine = ElasticSearchVectorSearchEngine(es_client)
search_engine.search(v, num_results=5)

[{'question': 'The course has already started. Can I still join it?',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and 

In [33]:
search_engine = ElasticSearchVectorSearchEngine(es_client)
search_function = searcher(embedding_model, search_engine)

hit_rate_evaluator = metric_evaluator(hit_rate)
hit_rate_evaluator(ground_truth, lambda v: search_function(v['question'], num_results=5))

100%|██████████| 1830/1830 [01:11<00:00, 25.64it/s]


0.9398907103825137